In [1]:
from bayescache.data import P3B3
from bayescache.api.data import train_valid_split
from bayescache.api.data.source import TrainingData

from torch.utils.data import DataLoader

In [19]:
train = P3B3('/Users/yngtodd/data', partition='train', download=True)

In [27]:
train

Dataset P3B3
    Number of datapoints: 8000
    Split: train
    Root Location: /Users/yngtodd/data

In [28]:
labels = train.targets

In [29]:
print(labels.shape)
targets = {
    'subsite': labels[:,0],
    'laterality': labels[:,1],
    'behavior': labels[:,2],
    'grade': labels[:,3]
}

(8000, 4)


In [33]:
print(targets['subsite'].shape)
print(targets['subsite'])

(8000,)
[5 4 5 ... 5 5 5]


In [35]:
targets['subsite'][0:10]

array([5, 4, 5, 5, 1, 0, 1, 5, 5, 1])

In [36]:
subsite_batch = targets['subsite'][0:10],
laterality_batch = targets['laterality'][0:10],
behavior_batch = targets['behavior'][0:10],
grade_batch = targets['grade'][0:10],

batch = {
    'subsite': subsite_batch,
    'laterality': laterality_batch,
    'behavior': behavior_batch,
    'grade': grade_batch  
}

In [37]:
batch

{'subsite': (array([5, 4, 5, 5, 1, 0, 1, 5, 5, 1]),),
 'laterality': (array([0, 0, 0, 1, 1, 1, 1, 0, 0, 0]),),
 'behavior': (array([2, 2, 1, 2, 2, 2, 2, 1, 1, 2]),),
 'grade': (array([2, 1, 0, 2, 2, 2, 2, 1, 1, 2]),)}

In [4]:
train, valid = train_valid_split(train, valpercent=.25)

In [5]:
len(train)

6000

In [6]:
len(valid)

2000

In [10]:
traindata = TrainingData(train, .25, 2, 1)

In [11]:
val_loader = traindata.val_loader()

In [12]:
for idx, (data, label) in enumerate(val_loader):
    print(f'idx: {idx}, label: {label}')

idx: 0, label: tensor([[3, 0, 2, 1]])
idx: 1, label: tensor([[0, 0, 2, 2]])
idx: 2, label: tensor([[5, 1, 2, 1]])
idx: 3, label: tensor([[1, 1, 2, 2]])
idx: 4, label: tensor([[0, 0, 2, 2]])
idx: 5, label: tensor([[5, 1, 2, 2]])
idx: 6, label: tensor([[5, 1, 2, 0]])
idx: 7, label: tensor([[5, 0, 2, 1]])
idx: 8, label: tensor([[5, 1, 2, 2]])
idx: 9, label: tensor([[5, 1, 2, 2]])
idx: 10, label: tensor([[5, 0, 2, 2]])
idx: 11, label: tensor([[5, 0, 1, 1]])
idx: 12, label: tensor([[5, 1, 2, 1]])
idx: 13, label: tensor([[5, 0, 2, 1]])
idx: 14, label: tensor([[5, 0, 1, 2]])
idx: 15, label: tensor([[5, 1, 2, 2]])
idx: 16, label: tensor([[4, 1, 2, 1]])
idx: 17, label: tensor([[1, 1, 2, 2]])
idx: 18, label: tensor([[5, 1, 2, 2]])
idx: 19, label: tensor([[5, 1, 2, 2]])
idx: 20, label: tensor([[0, 1, 2, 2]])
idx: 21, label: tensor([[5, 0, 2, 2]])
idx: 22, label: tensor([[5, 0, 2, 1]])
idx: 23, label: tensor([[3, 0, 2, 1]])
idx: 24, label: tensor([[2, 0, 2, 2]])
idx: 25, label: tensor([[4, 1, 2, 1

idx: 279, label: tensor([[5, 0, 2, 1]])
idx: 280, label: tensor([[3, 0, 2, 1]])
idx: 281, label: tensor([[4, 0, 2, 1]])
idx: 282, label: tensor([[0, 0, 2, 2]])
idx: 283, label: tensor([[4, 1, 2, 1]])
idx: 284, label: tensor([[0, 1, 2, 2]])
idx: 285, label: tensor([[2, 1, 2, 2]])
idx: 286, label: tensor([[5, 0, 1, 0]])
idx: 287, label: tensor([[0, 0, 2, 1]])
idx: 288, label: tensor([[5, 1, 2, 0]])
idx: 289, label: tensor([[5, 1, 2, 1]])
idx: 290, label: tensor([[1, 0, 2, 2]])
idx: 291, label: tensor([[5, 0, 1, 2]])
idx: 292, label: tensor([[0, 0, 2, 2]])
idx: 293, label: tensor([[5, 0, 2, 1]])
idx: 294, label: tensor([[2, 0, 2, 2]])
idx: 295, label: tensor([[2, 1, 2, 2]])
idx: 296, label: tensor([[4, 0, 2, 1]])
idx: 297, label: tensor([[0, 1, 2, 1]])
idx: 298, label: tensor([[0, 0, 2, 2]])
idx: 299, label: tensor([[0, 0, 2, 1]])
idx: 300, label: tensor([[3, 0, 2, 1]])
idx: 301, label: tensor([[5, 0, 2, 2]])
idx: 302, label: tensor([[0, 1, 2, 1]])
idx: 303, label: tensor([[5, 0, 2, 2]])


idx: 550, label: tensor([[0, 1, 2, 2]])
idx: 551, label: tensor([[5, 1, 2, 1]])
idx: 552, label: tensor([[4, 0, 2, 1]])
idx: 553, label: tensor([[0, 1, 2, 1]])
idx: 554, label: tensor([[5, 1, 2, 1]])
idx: 555, label: tensor([[5, 0, 2, 2]])
idx: 556, label: tensor([[5, 1, 2, 2]])
idx: 557, label: tensor([[0, 0, 2, 2]])
idx: 558, label: tensor([[5, 1, 2, 2]])
idx: 559, label: tensor([[5, 0, 1, 2]])
idx: 560, label: tensor([[5, 0, 2, 2]])
idx: 561, label: tensor([[2, 1, 2, 2]])
idx: 562, label: tensor([[5, 0, 1, 2]])
idx: 563, label: tensor([[1, 1, 2, 2]])
idx: 564, label: tensor([[0, 0, 2, 2]])
idx: 565, label: tensor([[5, 1, 2, 0]])
idx: 566, label: tensor([[5, 1, 2, 1]])
idx: 567, label: tensor([[5, 0, 1, 0]])
idx: 568, label: tensor([[5, 0, 1, 0]])
idx: 569, label: tensor([[4, 1, 2, 1]])
idx: 570, label: tensor([[0, 1, 2, 1]])
idx: 571, label: tensor([[3, 0, 2, 1]])
idx: 572, label: tensor([[5, 0, 2, 1]])
idx: 573, label: tensor([[5, 1, 2, 2]])
idx: 574, label: tensor([[5, 0, 2, 2]])


idx: 840, label: tensor([[0, 0, 2, 2]])
idx: 841, label: tensor([[5, 1, 2, 0]])
idx: 842, label: tensor([[4, 0, 2, 1]])
idx: 843, label: tensor([[5, 0, 1, 0]])
idx: 844, label: tensor([[2, 0, 2, 2]])
idx: 845, label: tensor([[1, 1, 2, 2]])
idx: 846, label: tensor([[5, 0, 2, 0]])
idx: 847, label: tensor([[0, 1, 2, 2]])
idx: 848, label: tensor([[5, 0, 2, 2]])
idx: 849, label: tensor([[5, 1, 2, 2]])
idx: 850, label: tensor([[5, 0, 2, 1]])
idx: 851, label: tensor([[0, 0, 2, 2]])
idx: 852, label: tensor([[5, 0, 1, 1]])
idx: 853, label: tensor([[5, 0, 1, 2]])
idx: 854, label: tensor([[1, 1, 2, 2]])
idx: 855, label: tensor([[5, 0, 2, 0]])
idx: 856, label: tensor([[5, 0, 2, 2]])
idx: 857, label: tensor([[5, 1, 2, 2]])
idx: 858, label: tensor([[5, 1, 2, 2]])
idx: 859, label: tensor([[1, 0, 2, 2]])
idx: 860, label: tensor([[5, 0, 2, 1]])
idx: 861, label: tensor([[1, 1, 2, 2]])
idx: 862, label: tensor([[0, 1, 2, 2]])
idx: 863, label: tensor([[5, 0, 2, 1]])
idx: 864, label: tensor([[5, 0, 2, 2]])


idx: 1148, label: tensor([[0, 0, 2, 1]])
idx: 1149, label: tensor([[5, 1, 2, 0]])
idx: 1150, label: tensor([[5, 1, 2, 1]])
idx: 1151, label: tensor([[5, 0, 2, 2]])
idx: 1152, label: tensor([[3, 0, 2, 1]])
idx: 1153, label: tensor([[2, 1, 2, 2]])
idx: 1154, label: tensor([[5, 0, 1, 2]])
idx: 1155, label: tensor([[5, 0, 1, 1]])
idx: 1156, label: tensor([[1, 1, 2, 2]])
idx: 1157, label: tensor([[5, 1, 2, 0]])
idx: 1158, label: tensor([[1, 1, 2, 2]])
idx: 1159, label: tensor([[5, 0, 2, 2]])
idx: 1160, label: tensor([[5, 0, 2, 2]])
idx: 1161, label: tensor([[4, 1, 2, 1]])
idx: 1162, label: tensor([[4, 1, 2, 1]])
idx: 1163, label: tensor([[5, 0, 1, 0]])
idx: 1164, label: tensor([[0, 1, 2, 2]])
idx: 1165, label: tensor([[1, 0, 2, 2]])
idx: 1166, label: tensor([[1, 1, 2, 2]])
idx: 1167, label: tensor([[5, 1, 2, 2]])
idx: 1168, label: tensor([[5, 1, 2, 0]])
idx: 1169, label: tensor([[5, 0, 2, 1]])
idx: 1170, label: tensor([[5, 0, 2, 2]])
idx: 1171, label: tensor([[5, 0, 2, 1]])
idx: 1172, label

idx: 1435, label: tensor([[5, 1, 2, 2]])
idx: 1436, label: tensor([[5, 0, 1, 0]])
idx: 1437, label: tensor([[5, 1, 2, 2]])
idx: 1438, label: tensor([[0, 0, 2, 2]])
idx: 1439, label: tensor([[5, 1, 2, 2]])
idx: 1440, label: tensor([[5, 1, 2, 1]])
idx: 1441, label: tensor([[0, 1, 2, 2]])
idx: 1442, label: tensor([[3, 0, 2, 1]])
idx: 1443, label: tensor([[0, 0, 2, 2]])
idx: 1444, label: tensor([[3, 0, 2, 1]])
idx: 1445, label: tensor([[0, 1, 2, 1]])
idx: 1446, label: tensor([[5, 0, 1, 0]])
idx: 1447, label: tensor([[5, 0, 2, 0]])
idx: 1448, label: tensor([[5, 0, 1, 1]])
idx: 1449, label: tensor([[0, 0, 2, 2]])
idx: 1450, label: tensor([[0, 1, 2, 2]])
idx: 1451, label: tensor([[5, 1, 2, 2]])
idx: 1452, label: tensor([[2, 1, 2, 2]])
idx: 1453, label: tensor([[5, 1, 2, 1]])
idx: 1454, label: tensor([[5, 1, 2, 1]])
idx: 1455, label: tensor([[2, 1, 2, 2]])
idx: 1456, label: tensor([[5, 0, 1, 1]])
idx: 1457, label: tensor([[5, 1, 2, 2]])
idx: 1458, label: tensor([[5, 0, 1, 2]])
idx: 1459, label

In [13]:
valid = traindata.val_dataset()

In [14]:
len(valid)

1500

In [15]:
train = traindata.train_dataset()

In [16]:
len(train)

4500

In [16]:
source = traindata.train_loader()

Dataset P3B3
    Number of datapoints: 8000
    Split: train
    Root Location: /Users/yngtodd/data